# Big Data - Movie Recommender System

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml.feature import  Imputer, StringIndexer, VectorAssembler
from pyspark.ml.linalg import SparseVector, DenseVector
from pyspark.sql.types import *
from pyspark.ml import Pipeline
from pyspark.ml.recommendation import ALS # Alternating least square  
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder, CrossValidator, TrainValidationSplit
from pyspark.ml.evaluation import RegressionEvaluator


In [0]:
#instantiate the spark session
spark = SparkSession.builder.appName("A2-PartB").getOrCreate()
 
#set the shuffle partition same as number of cpu cores to improve performance 
# spark.conf.set("spark.sql.shuffle.partitions")

In [0]:
# File location and type
file_location = "/FileStore/tables/movies.csv"
file_type = "csv"

# CSV options
infer_schema = "True"
first_row_is_header = "True"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)


movieId,rating,userId
2,3,0
3,1,0
5,2,0
9,4,0
11,1,0
12,2,0
15,1,0
17,1,0
19,1,0
21,1,0


In [0]:
df.rdd.getNumPartitions()


Out[4]: 1

In [0]:
df.printSchema()


root
-- movieId: integer (nullable = true)
-- rating: integer (nullable = true)
-- userId: integer (nullable = true)

In [0]:
# check the count of null values for each column
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+-------+------+------+
movieId|rating|userId|
+-------+------+------+
 0| 0| 0|
+-------+------+------+

In [0]:
rows = df.count()
columns = len(df.columns)
shape = (rows,columns)
print(shape)

(1501, 3)

In [0]:
# Looking at top 10 Movies which recieved the most ratings
q1a = df.select('movieId','rating').groupby('movieId').agg(count('rating').alias('count')).orderBy(desc('count')).limit(10)
q1a.show()


+-------+-----+
movieId|count|
+-------+-----+
 51| 20|
 6| 20|
 50| 20|
 22| 20|
 29| 20|
 94| 19|
 55| 19|
 15| 19|
 68| 19|
 2| 19|
+-------+-----+

In [0]:
# Looking at top 10 Movies based on highest count of highest rating they have recieved
Extra1  = df.select('movieId','rating').groupby('movieId','rating').agg(count('*').alias('count')).orderBy(desc('rating'),desc('count')).limit(10)
Extra1.show()


+-------+------+-----+
movieId|rating|count|
+-------+------+-----+
 90| 5| 3|
 51| 5| 3|
 55| 5| 3|
 46| 5| 3|
 30| 5| 3|
 52| 5| 3|
 27| 5| 3|
 22| 5| 3|
 29| 5| 3|
 23| 5| 2|
+-------+------+-----+

In [0]:
# fltr = (df['movieID'] == 2)

# df.where(fltr).show()

In [0]:
# Looking at top 10 Users who have provided the most ratings
q1b = df.select('userId','rating').groupby('userId').agg(count('rating').alias('count')).orderBy(desc('count')).limit(10)
q1b.show()


+------+-----+
userId|count|
+------+-----+
 6| 57|
 14| 57|
 22| 56|
 11| 56|
 12| 55|
 4| 55|
 7| 54|
 9| 53|
 23| 52|
 24| 52|
+------+-----+

In [0]:
# Looking at Users who have provided the highest rating max amount of time

Extra2 = df.select('userId','rating').groupby('userId','rating').agg(count('*').alias('count')).orderBy(desc('rating'),desc('count')).limit(10)
Extra2.show()


+------+------+-----+
userId|rating|count|
+------+------+-----+
 11| 5| 8|
 26| 5| 6|
 23| 5| 6|
 22| 5| 6|
 2| 5| 5|
 24| 5| 5|
 17| 5| 5|
 12| 5| 4|
 8| 5| 4|
 18| 5| 4|
+------+------+-----+

In [0]:
(training1, test1) = df.randomSplit([0.7, 0.3])
(training2, test2) = df.randomSplit([0.8, 0.2])

In [0]:
MOVIEID = 'movieId'
RATING = 'rating'
USERID = 'userId'

als = ALS(maxIter=15 , regParam=0.01, userCol=USERID, itemCol=MOVIEID, ratingCol=RATING)


In [0]:
model1 = als.fit(training1)

predictions1 = model1.transform(test1)
display(predictions1)

movieId,rating,userId,prediction
31,1,4,5.1812396
31,3,14,3.7451963
31,1,29,0.3347944
85,3,1,0.06412296
85,1,4,1.9018631
85,1,5,1.3831215
85,3,6,3.5036194
85,5,8,3.0272744
85,3,21,1.1056397
85,1,26,2.3982472


In [0]:
model2 = als.fit(training2)

predictions2 = model2.transform(test2)
display(predictions2)

movieId,rating,userId,prediction
31,1,0,3.2766697
31,1,13,1.0848343
31,1,19,2.4677005
31,1,27,0.6288843
31,1,29,1.5578259
85,1,2,-1.682748
85,1,15,0.21277595
85,2,20,-0.12727948
85,3,21,1.8258545
65,1,2,1.866737


In [0]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator_rmse = RegressionEvaluator(
           metricName="rmse", 
           labelCol=RATING, 
           predictionCol="prediction") 
evaluator_mse = RegressionEvaluator(
           metricName="mse", 
           labelCol=RATING, 
           predictionCol="prediction") 
evaluator_mae = RegressionEvaluator(
           metricName="mae", 
           labelCol=RATING, 
           predictionCol="prediction") 


In [0]:
RMSE1 = evaluator_rmse.evaluate(predictions1)
print("RMSE for Model 1: ", RMSE1)
MSE1 = evaluator_mse.evaluate(predictions1)
print("MSE for Model 1: ", MSE1)
MAE1 = evaluator_mae.evaluate(predictions1)
print("MAE for Model 1: ", MAE1)

RMSE for Model 1: 1.7261676062688378
MSE for Model 1: 2.9796546049318895
MAE for Model 1: 1.2993274276356466

In [0]:
RMSE2 = evaluator_rmse.evaluate(predictions2)
print("RMSE for Model 2: ", RMSE2)
MSE2 = evaluator_mse.evaluate(predictions2)
print("MSE for Model 2: ", MSE2)
MAE2 = evaluator_mae.evaluate(predictions2)
print("MAE for Model 2: ", MAE2)

RMSE for Model 2: 1.5757532774534835
MSE for Model 2: 2.482998391405395
MAE for Model 2: 1.1791483546778756

Looking at the results above we can see that model 2 performed better with lower RMSE and MSE which could be as due to the higer training set data. in model 1 (70-30) split of the dataset was not enough and the model could be slightly underfitting causing higher bias which results in higher error. 

However, mean absolute  error (MAE) for both model is similar.

We will be using cross validation and along with gridsearch for finding the best parameters

In [0]:
als = ALS(userCol=USERID, itemCol=MOVIEID, ratingCol=RATING)

In [0]:
param_grid = ParamGridBuilder() \
            .addGrid(als.rank, [50, 100]) \
            .addGrid(als.maxIter, [15, 25])\
            .addGrid(als.regParam, [.01, .1]) \
            .build()

#### Using Cross Validation

In [0]:
# Using Corss Validation
cv = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator_rmse, numFolds=2)

In [0]:
# (training2, test2) = df.randomSplit([0.8, 0.2])
cv_model = cv.fit(training2)

/databricks/spark/python/pyspark/ml/util.py:800: UserWarning: Cannot find mlflow module. To enable MLflow logging, install mlflow from PyPI.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)

In [0]:
#Extract best model from the cv model above
best_model = cv_model.bestModel


In [0]:
# Print "Rank"
print("  Rank:", best_model._java_obj.parent().getRank())
# Print "MaxIter"
print("  MaxIter:", best_model._java_obj.parent().getMaxIter())
# Print "RegParam"
print("  RegParam:", best_model._java_obj.parent().getRegParam())

Rank: 100
 MaxIter: 25
 RegParam: 0.1

In [0]:
# View the predictions
test2_predictions_tv = best_model.transform(test2)
RMSE_cv = evaluator_rmse.evaluate(test2_predictions_tv)
print("RMSE Value is: ", RMSE_cv)

RMSE Value is: 1.0011431268537268

In [0]:
display(test2_predictions_tv)

movieId,rating,userId,prediction
31,1,0,1.1813673
31,1,13,1.4173347
31,1,19,1.2622163
31,1,27,1.0882876
31,1,29,1.68816
85,1,2,1.1283349
85,1,15,0.98939586
85,2,20,2.3875895
85,3,21,2.6229606
65,1,2,1.0328238


In [0]:
trainvs = TrainValidationSplit(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator_rmse)

In [0]:
tv_model = trainvs.fit(training2)

In [0]:
#Extract best model from the cv model above
best_model_tV = tv_model.bestModel


In [0]:
# Print "Rank"
print("  Rank:", best_model_tV._java_obj.parent().getRank())
# Print "MaxIter"
print("  MaxIter:", best_model_tV._java_obj.parent().getMaxIter())
# Print "RegParam"
print("  RegParam:", best_model_tV._java_obj.parent().getRegParam())

Rank: 50
 MaxIter: 25
 RegParam: 0.1

In [0]:
# View the predictions
test2_predictions_tv = best_model_tV.transform(test2)
RMSE_tv = evaluator_rmse.evaluate(test2_predictions_tv)
print("RMSE Value is: ", RMSE_tv)

RMSE Value is: 1.001091579255891

In [0]:
display(test2_predictions_tv)

movieId,rating,userId,prediction
31,1,0,1.1813673
31,1,13,1.4173347
31,1,19,1.2622163
31,1,27,1.0882876
31,1,29,1.68816
85,1,2,1.1283349
85,1,15,0.98939586
85,2,20,2.3875895
85,3,21,2.6229606
65,1,2,1.0328238


# QUESTION 5: Recommendation Using Cross Validation

In [0]:

movie_recommendations = best_model.recommendForAllUsers(60)
movie_recommendations.show()

+------+--------------------+
userId| recommendations|
+------+--------------------+
 20|[[22, 4.022359], ...|
 10|[[2, 3.2582815], ...|
 0|[[2, 2.522292], [...|
 1|[[68, 3.2300415],...|
 21|[[52, 4.220557], ...|
 11|[[18, 4.7560287],...|
 12|[[46, 4.6036544],...|
 22|[[75, 4.488112], ...|
 2|[[93, 4.632242], ...|
 13|[[93, 3.3210738],...|
 3|[[51, 4.066766], ...|
 23|[[32, 4.79486], [...|
 4|[[29, 3.4332924],...|
 24|[[30, 4.5147476],...|
 14|[[52, 4.638243], ...|
 5|[[55, 3.9734645],...|
 15|[[46, 4.171382], ...|
 25|[[47, 3.2089407],...|
 26|[[22, 4.6670384],...|
 6|[[58, 2.8545136],...|
+------+--------------------+
only showing top 20 rows

In [0]:
# display(movie_recommendations)

In [0]:
cv_recommendations = movie_recommendations\
    .withColumn("rec_exp", explode("recommendations"))\
    .select('userId', col("rec_exp.movieId"), col("rec_exp.rating"))

In [0]:
cv_recommendations.show()

+------+-------+---------+
userId|movieId| rating|
+------+-------+---------+
 20| 22| 4.022359|
 20| 94|3.3820114|
 20| 77|3.3124428|
 20| 75| 3.290255|
 20| 68|3.2445157|
 20| 88|3.0268147|
 20| 98| 2.971247|
 20| 51|2.9312098|
 20| 53|2.7102458|
 20| 90|2.6760623|
 20| 62|2.6663806|
 20| 74| 2.475801|
 20| 54|2.4504795|
 20| 30|2.4443252|
 20| 85| 2.387853|
 20| 32|2.2987494|
 20| 20| 2.137775|
 20| 24|2.1294396|
 20| 46|2.0688202|
 20| 36|2.0685816|
+------+-------+---------+
only showing top 20 rows

In [0]:
cv_filter = cv_recommendations['userId'] == 11

cv_recommendations_11 = cv_recommendations.where(cv_filter)
cv_recommendations_11.show()

+------+-------+---------+
userId|movieId| rating|
+------+-------+---------+
 11| 18|4.7560287|
 11| 30| 4.728299|
 11| 32|4.6885595|
 11| 23|4.5803294|
 11| 27| 4.505392|
 11| 48| 4.404255|
 11| 79| 4.151344|
 11| 13|3.6438527|
 11| 90|3.6310692|
 11| 49|3.5694964|
 11| 19|3.4558816|
 11| 81|3.4252162|
 11| 66|3.3622818|
 11| 50|3.3575008|
 11| 55|3.3267386|
 11| 8| 3.193362|
 11| 38| 3.114695|
 11| 80|2.9385731|
 11| 51|2.8964515|
 11| 93|2.8043761|
+------+-------+---------+
only showing top 20 rows

In [0]:
# Movies already watched by User 11
watched_user11 = df.select(['userId','movieId',  'rating']).filter(df.userId == 11).orderBy('rating', ascending = False)
watched_user11.show()

+------+-------+------+
userId|movieId|rating|
+------+-------+------+
 11| 18| 5|
 11| 48| 5|
 11| 27| 5|
 11| 69| 5|
 11| 79| 5|
 11| 23| 5|
 11| 30| 5|
 11| 32| 5|
 11| 50| 4|
 11| 66| 4|
 11| 81| 4|
 11| 19| 4|
 11| 90| 4|
 11| 13| 4|
 11| 38| 4|
 11| 35| 3|
 11| 51| 3|
 11| 71| 3|
 11| 75| 3|
 11| 72| 3|
+------+-------+------+
only showing top 20 rows

In [0]:
# Extracting movie id for watched movies
movie_list = []
for movie in watched_user11.collect():
  movie_list.append(movie.movieId)

In [0]:
print(len(movie_list))

56

In [0]:
Unwatched_recommended_user11 = (cv_recommendations_11.filter(cv_recommendations_11.movieId.isin(movie_list) == False))
Unwatched_recommended_user11.show()

+------+-------+---------+
userId|movieId| rating|
+------+-------+---------+
 11| 49|3.5694964|
 11| 55|3.3267386|
 11| 8| 3.193362|
 11| 93|2.8043761|
 11| 34|2.5651197|
 11| 46|2.4032905|
 11| 7|2.3851585|
 11| 83|2.3656769|
 11| 73| 2.345943|
 11| 17|2.1344254|
 11| 87|2.1150675|
 11| 44|2.1079774|
 11| 63| 2.052105|
 11| 96|1.9910831|
 11| 54| 1.935037|
 11| 24|1.9124683|
 11| 65|1.8602684|
 11| 74|1.7630874|
 11| 68|1.7236478|
 11| 31|1.7071593|
+------+-------+---------+
only showing top 20 rows

In [0]:
print(Unwatched_recommended_user11.count())

29

In [0]:
Unwatched_recommended_user11.orderBy(desc('rating')).limit(15).show()

+------+-------+---------+
userId|movieId| rating|
+------+-------+---------+
 11| 49|3.5694964|
 11| 55|3.3267386|
 11| 8| 3.193362|
 11| 93|2.8043761|
 11| 34|2.5651197|
 11| 46|2.4032905|
 11| 7|2.3851585|
 11| 83|2.3656769|
 11| 73| 2.345943|
 11| 17|2.1344254|
 11| 87|2.1150675|
 11| 44|2.1079774|
 11| 63| 2.052105|
 11| 96|1.9910831|
 11| 54| 1.935037|
+------+-------+---------+

In [0]:
cv_filter2 = cv_recommendations['userId'] == 23

cv_recommendations_23 = cv_recommendations.where(cv_filter2)
cv_recommendations_23.show()

+------+-------+---------+
userId|movieId| rating|
+------+-------+---------+
 23| 32| 4.79486|
 23| 55| 4.673742|
 23| 49|4.5430574|
 23| 48|4.3785806|
 23| 90| 4.198768|
 23| 27|3.9622667|
 23| 23|3.8798094|
 23| 30|3.8768952|
 23| 17|3.8110025|
 23| 50|3.7684746|
 23| 7|3.6960728|
 23| 18|3.6585617|
 23| 46|3.6053803|
 23| 13| 3.552243|
 23| 64|3.5466485|
 23| 94|3.1548033|
 23| 35|3.0985942|
 23| 68|3.0275335|
 23| 20|2.7890196|
 23| 87|2.7621088|
+------+-------+---------+
only showing top 20 rows

In [0]:
# Movies already watched by User 23
watched_user23 = df.select(['userId','movieId',  'rating']).filter(df.userId == 23).orderBy('rating', ascending = False)
# Extracting movie id for watched movies
movie_list23 = []
for movie in watched_user23.collect():
  movie_list23.append(movie.movieId)
print(len(movie_list23))

52

In [0]:
Unwatched_recommended_user23 = (cv_recommendations_23.filter(cv_recommendations_23.movieId.isin(movie_list23) == False))
Unwatched_recommended_user23.orderBy(desc('rating')).limit(15).show()

+------+-------+---------+
userId|movieId| rating|
+------+-------+---------+
 23| 90| 4.198768|
 23| 17|3.8110025|
 23| 7|3.6960728|
 23| 46|3.6053803|
 23| 94|3.1548033|
 23| 35|3.0985942|
 23| 20|2.7890196|
 23| 79|2.6687233|
 23| 56|2.5495627|
 23| 81|2.4428444|
 23| 19|2.2522676|
 23| 16|2.1279554|
 23| 91|2.0733461|
 23| 80|2.0283413|
 23| 75|1.9893386|
+------+-------+---------+

In [0]:
watched_user11 = df.select(['userId','movieId',  'rating']).filter(df.userId == 11).orderBy('rating', ascending = False)

In [0]:
# MOVIES ALREADY WATCHED BY THE USER
watched_user11.show()

+------+-------+------+
userId|movieId|rating|
+------+-------+------+
 11| 18| 5|
 11| 48| 5|
 11| 27| 5|
 11| 69| 5|
 11| 79| 5|
 11| 23| 5|
 11| 30| 5|
 11| 32| 5|
 11| 50| 4|
 11| 66| 4|
 11| 81| 4|
 11| 19| 4|
 11| 90| 4|
 11| 13| 4|
 11| 38| 4|
 11| 35| 3|
 11| 51| 3|
 11| 71| 3|
 11| 75| 3|
 11| 72| 3|
+------+-------+------+
only showing top 20 rows

In [0]:
# Extracting movie id for watched movies
movie_list = []
for movie in watched_user11.collect():
  movie_list.append(movie.movieId)

In [0]:
Unwatched_user11 = (df.filter(df.movieId.isin(movie_list) == False))

In [0]:
Unwatched_user11.select('movieId','rating').show()

+-------+------+
movieId|rating|
+-------+------+
 2| 3|
 3| 1|
 5| 2|
 15| 1|
 17| 1|
 26| 3|
 28| 1|
 29| 1|
 31| 1|
 34| 1|
 44| 1|
 46| 1|
 54| 1|
 55| 1|
 68| 1|
 83| 1|
 87| 1|
 91| 3|
 92| 4|
 95| 2|
+-------+------+
only showing top 20 rows

In [0]:
# Prediction based on Best ALS model from Cross Validation
movies_15_recommendation_user_11 = best_model.transform(Unwatched_user11)
display(movies_15_recommendation_user_11.select('movieId','prediction'))

movieId,prediction
31,1.1811305
31,1.2988654
31,1.2270397
31,2.6040952
31,2.7394843
31,3.3450334
31,1.417555
31,2.5627148
31,1.0549653
31,1.2622926


In [0]:
top15Movies = movies_15_recommendation_user_11.select('movieId','prediction').orderBy('prediction', ascending = False).withColumn('userId', lit(11)).show(15)


+-------+----------+------+
movieId|prediction|userId|
+-------+----------+------+
 46| 4.726519| 11|
 55| 4.673742| 11|
 52| 4.638243| 11|
 93| 4.632242| 11|
 83| 4.6114244| 11|
 49| 4.5430574| 11|
 52| 4.5407763| 11|
 29| 4.4785304| 11|
 52| 4.452403| 11|
 24| 4.387907| 11|
 8| 4.3835964| 11|
 83| 4.366677| 11|
 55| 4.357387| 11|
 96| 4.349066| 11|
 17| 4.292567| 11|
+-------+----------+------+
only showing top 15 rows

In [0]:
watched_user23 = df.select(['userId','movieId',  'rating']).filter(df.userId == 23).orderBy('rating', ascending = False)
watched_user23.select('movieId','rating').show()

+-------+------+
movieId|rating|
+-------+------+
 65| 5|
 27| 5|
 48| 5|
 49| 5|
 32| 5|
 55| 5|
 64| 4|
 10| 4|
 13| 4|
 30| 4|
 50| 4|
 18| 4|
 23| 4|
 68| 3|
 36| 3|
 73| 3|
 87| 3|
 6| 2|
 22| 2|
 33| 2|
+-------+------+
only showing top 20 rows

In [0]:
# Extracting movie id for watched movies
movie_list_23 = []
for movie in watched_user23.collect():
  movie_list_23.append(movie.movieId)

In [0]:
# Getting Unwatched movies
Unwatched_user23 = (df.filter(df.movieId.isin(movie_list_23) == False))

movies_15_recommendation_user_23 = best_model.transform(Unwatched_user23)
display(movies_15_recommendation_user_23.select('movieId','prediction'))

movieId,prediction
31,1.1811305
31,1.2988654
31,1.2270397
31,2.6040952
31,2.7394843
31,3.3450334
31,1.417555
31,2.5627148
31,1.0549653
31,1.2622926


In [0]:
top15Movies_23 = movies_15_recommendation_user_23.select('movieId','prediction').orderBy('prediction', ascending = False).withColumn('userId', lit(23)).show(15)


+-------+----------+------+
movieId|prediction|userId|
+-------+----------+------+
 46| 4.726519| 23|
 52| 4.638243| 23|
 93| 4.632242| 23|
 90| 4.586747| 23|
 94| 4.5454707| 23|
 52| 4.5407763| 23|
 75| 4.488112| 23|
 52| 4.452403| 23|
 8| 4.3835964| 23|
 96| 4.349066| 23|
 90| 4.3384457| 23|
 90| 4.2941847| 23|
 17| 4.292567| 23|
 51| 4.2914286| 23|
 46| 4.2725654| 23|
+-------+----------+------+
only showing top 15 rows